In [ ]:
print("OK")

In [ ]:
%pwd

In [ ]:
import os 
os.chdir("../")

In [ ]:
%pwd

In [ ]:
# import sys
# print(sys.executable)


In [ ]:
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.embeddings import SentenceTransformerEmbeddings

print("LangChain 0.3.x stack OK ✅")


In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# Extract text from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [ ]:
extracted_data = load_pdf_files(r"C:\Users\kanha\Building-medical-chat-bot\data")

In [ ]:
extracted_data


In [ ]:
len(extracted_data)

In [ ]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [ ]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [ ]:
minimal_docs

In [ ]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [ ]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

In [ ]:
texts_chunk

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

In [ ]:
embedding

In [ ]:
vector = embedding.embed_query("Hello world")
vector

In [ ]:
print( "Vector length:", len(vector))

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [ ]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
import os

os.environ["OPENAI_API_KEY"] = "*****************************"
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"


In [62]:
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [63]:
pc

In [64]:
from pinecone import ServerlessSpec 

index_name = "medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [ ]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)

In [65]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

# Add more data to the existing Pinecone index

In [ ]:
dswith = Document(
    page_content="dswithbappy is a youtube channel that provides tutorials on various topics.",
    metadata={"source": "Youtube"}
)

In [ ]:
docsearch.add_documents(documents=[dswith])

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [ ]:
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

In [ ]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="llama-3.1-8b-instant"
)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="llama-3.1-8b-instant",
    temperature=0.3
)

print(llm.invoke("Say hello").content)


In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
system_prompt = (
    "You are an Medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "*****************************"
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"


In [ ]:
import os

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [ ]:
print(os.environ["OPENAI_API_KEY"][:6])
print(os.environ["OPENAI_API_BASE"])


In [ ]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

In [66]:
response = rag_chain.invoke({"input": "what is scar?"})
print(response["answer"])

A scar is a permanent mark on the skin resulting from the healing process of a wound, cut, or injury. It occurs when the body repairs damaged tissue by forming new collagen fibers. Scars can be raised, flat, or have different textures and may have a different color compared to the surrounding skin.


In [67]:
response = rag_chain.invoke({"input": "what is the Treatment of Acne?"})
print(response["answer"])

The treatment of acne typically involves a combination of topical and oral medications, as well as lifestyle modifications. Topical treatments, such as benzoyl peroxide and retinoids, can help reduce bacteria and inflammation on the skin's surface. Oral antibiotics and retinoids may also be prescribed to treat more severe cases, although long-term use should be monitored due to potential side effects.


In [68]:
response = rag_chain.invoke({"input": "what dswithbappy?"})
print(response["answer"])

I don't have enough information to answer that question.
